# Quantum Classification With Feature Engineering

This notebook offers several examples of feature engineering used with the [QML library](https://docs.microsoft.com/azure/quantum/user-guide/libraries/machine-learning/intro) and a set of exercises on picking the right parameters to distinguish classes of increasingly complex shapes.

* This Jupyter notebook can be executed on Azure Quantum.
* The first part of this tutorial repeats the steps of the [Exploring Quantum Classification Library tutorial](./ExploringQuantumClassificationLibrary.ipynb) with a slightly different data set for the sake of completeness when running on Azure Quantum.
* The companion Q# notebook [Inside Quantum Classifiers](./InsideQuantumClassifiers.ipynb) offers a deep dive in the internals of a simple quantum classifier and several exercises on implementing it from scratch.

## Setup

To start with, execute this cell using Ctrl+Enter (or ⌘+Enter on a Mac). This is necessary to prepare the environment, import the Q# libraries and operations we'll use later in the tutorial, and configure the plotting routines. If any Python packages are reported as missing, install them.

In [ ]:
import math
import random
from typing import List

import numpy as np
from matplotlib import pyplot
pyplot.style.use('ggplot')

import warnings
warnings.simplefilter('ignore')

%matplotlib inline

# Plotting configuration
cases = [(0, 0), (0, 1), (1, 1), (1, 0)]
markers = [
    '.' if actual == classified else 'X'
    for (actual, classified) in cases
]
colors = ['blue', 'blue', 'red', 'red']

# Q# configuration and necessary imports
import qsharp
qsharp.packages.add("Microsoft.Quantum.MachineLearning")
qsharp.reload()

print()
print("Setup complete!")

## 1. Classes Separable by Lines That Go Through (0, 0)

### Artificial Data Generation

The first step of solving a classification problem is preparing the training and validation datasets.

> In this tutorial we will use artificially generated data of several different types.
A real classification problem will load real data instead, but this choice of artificial data allows to construct simple quantum classifiers by hand, which will be helpful for a deep dive in the classifier structure.

In [ ]:
def generate_angular_data(samples_number : int, min : float, max : float, separation_angles : List[float]):
    """Generates data with 2 features and 2 classes separable by a pair of lines that go through the origin"""
    features = []
    labels = []
    for i in range(samples_number):
        sample = [random.random() * (max - min) + min, random.random() * (max - min) + min]
        angle = math.atan(sample[1] / sample[0])
        features.append(sample)
        labels.append(0 if angle < separation_angles[0] or angle > separation_angles[1] else 1)
    
    data = { 'Features' : features, 'Labels' : labels }
    return data



In [ ]:
def plot_data(features : list, actual_labels : list, classified_labels : list = None, extra_lines : list = None):
    """Plots the data, labeling it with actual labels if there are no classification results provided, 
    and with the classification results (indicating their correctness) if they are provided.
    """
    samples = np.array(features)
    pyplot.figure(figsize=(8, 8))
    for (idx_case, ((actual, classified), marker, color)) in enumerate(zip(cases, markers, colors)):
        mask = np.logical_and(np.equal(actual_labels, actual), 
                              np.equal(actual if classified_labels == None else classified_labels, classified))
        if not np.any(mask): continue
        pyplot.scatter(
            samples[mask, 0], samples[mask, 1],
            label=f"Class {actual}" if classified_labels == None else f"Was {actual}, classified {classified}",
            marker=marker, s=300, c=[color],
        )
    # Add the lines to show the true classes boundaries, if provided
    if extra_lines != None:
        for line in extra_lines:
            pyplot.plot(line[0], line[1], color='gray')
    pyplot.legend()


### Q# Training and Classification Code

The Q# code in this cell wraps the calls to the training and classification APIs in the QML library. Later in the tutorial we'll use these wrappers to call the APIs for the specific model structures.

In [ ]:
%%qsharp
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.MachineLearning;
open Microsoft.Quantum.Math;

// The defaul schedule for drawing batches from a set of samples: just use all samples.
function DefaultSchedule(samples : Double[][]) : SamplingSchedule {
    return SamplingSchedule([
        0..Length(samples) - 1
    ]);
}


// Entry point for training a model; takes the data and the classifier structure as the input and produces model parameters and bias.
operation TrainClassifierModel(
    trainingVectors : Double[][],
    trainingLabels : Int[],
    classifierStructure : ControlledRotation[],
    initialParameters : Double[][]
) : (Double[], Double) {
    // convert training data and labels into a single data structure
    let samples = Mapped(
        LabeledSample,
        Zipped(trainingVectors, trainingLabels)
    );
    let (optimizedModel, nMisses) = TrainSequentialClassifier(
        Mapped(
            SequentialModel(classifierStructure, _, 0.0),
            initialParameters
        ),
        samples,
        DefaultTrainingOptions()
            w/ LearningRate <- 2.0
            w/ Tolerance <- 0.0005,
        DefaultSchedule(trainingVectors),
        DefaultSchedule(trainingVectors)
    );
    Message($"Training complete, found optimal parameters: {optimizedModel::Parameters}, {optimizedModel::Bias} with {nMisses} misses");
    return (optimizedModel::Parameters, optimizedModel::Bias);
}


// Entry point for using the model to classify the data; takes validation data, classifier structure, model parameters, and biad as inputs.
operation ClassifyData(
    samples : Double[][],
    classifierStructure : ControlledRotation[],
    parameters : Double[],
    bias : Double
) : Int[] {
    let tolerance = 0.0005;
    let nMeasurements = 10000;
    let model = Default<SequentialModel>()
        w/ Structure <- classifierStructure
        w/ Parameters <- parameters
        w/ Bias <- bias;
    let probabilities = EstimateClassificationProbabilities(
        tolerance, model,
        samples, nMeasurements
    );
    return InferredLabels(model::Bias, probabilities);
}


### Classifier Structure Selection

Selecting the right classifier structure requires some insight into the structure of the problem. In this case, we'll encode the data in a single qubit and use a model consisting of a single parameterized rotation gate $R_y(\theta)$.

In [ ]:
# Not required, but is helpful to provide type information to the Jupyter Notebooks IDE later in the code.
TrainModelAngularData: qsharp.QSharpCallable = None
ClassifyAngularData: qsharp.QSharpCallable = None
TrainModelVerticalData: qsharp.QSharpCallable = None
ClassifyVerticalData: qsharp.QSharpCallable = None
TrainModelHorizontalData: qsharp.QSharpCallable = None
ClassifyHorizontalData: qsharp.QSharpCallable = None
TrainModelCircleData: qsharp.QSharpCallable = None
ClassifyCircleData: qsharp.QSharpCallable = None
TrainModelHyperbolaData: qsharp.QSharpCallable = None
ClassifyHyperbolaData: qsharp.QSharpCallable = None

In [ ]:
%%qsharp
// The definition of classifier structure for the case when the data 
// is linearly separable by lines going through the origin and fits into 1 qubit.
function ClassifierStructureAngularData() : ControlledRotation[] {
    return [
        ControlledRotation((0, []), PauliY, 0)
    ];
}

// The training code for the case when the data is linearly separable by lines going through the origin.
operation TrainModelAngularData(
    trainingVectors : Double[][],
    trainingLabels : Int[],
    initialParameters : Double[][]
) : (Double[], Double) {
    return TrainClassifierModel(trainingVectors, trainingLabels, ClassifierStructureAngularData(), initialParameters);
}

// The classification code for the case when the data is linearly separable by lines going through the origin.
operation ClassifyAngularData(
    samples : Double[][],
    parameters : Double[],
    bias : Double
) : Int[] {
    return ClassifyData(samples, ClassifierStructureAngularData(), parameters, bias);
}


### Training and Validation

Now that the data generation, training, and classification code is ready, we can get to the interesting part: training the model and validating the results!

In [ ]:
# Generate training and validation data using the same pair of separation angles
separation_angles = [-math.pi / 4, math.pi / 4]
training_data_angular = generate_angular_data(150, -1, 1, separation_angles)
validation_data_angular = generate_angular_data(50, -1, 1, separation_angles)
print("Training and validation data generated")

def separation_line(angle : float):
    if angle < math.pi / 4:
        return [[-1, 1], [-math.tan(angle), math.tan(angle)]]
    return [[-1/math.tan(angle), 1/math.tan(angle)], [-1, 1]]

# Set up lines that show class separation
class_separation_lines_angular = list(map(separation_line, separation_angles))
    
plot_data(training_data_angular['Features'], training_data_angular['Labels'], extra_lines=class_separation_lines_angular)

In [ ]:
# Train the model
(parameters, bias) = TrainModelAngularData.simulate(
    trainingVectors=training_data_angular['Features'],
    trainingLabels=training_data_angular['Labels'],
    initialParameters=[[1.0], [2.0]]                   # use several parameter guesses to start training with
)

In [ ]:
# Classify validation data set using training results
classified_labels = ClassifyAngularData.simulate(
    samples=validation_data_angular['Features'],
    parameters=parameters, bias=bias
)

# Calculate miss rate
mask = np.not_equal(validation_data_angular['Labels'], classified_labels)
miss_count = np.array(classified_labels)[np.where(mask)].size
miss_rate = miss_count / len(classified_labels)
print(f"Miss rate: {miss_rate:0.2%}")

# Plot validation results
plot_data(validation_data_angular['Features'], 
          validation_data_angular['Labels'], classified_labels, class_separation_lines_angular)

## 2. Classes Separable by Vertical or Horizontal Lines

Let's now take a look at using quantum classifiers to distinguish classes of other shapes.

### 2.1. Classes Separable by a Vertical Line

In [ ]:
def generate_vertically_separated_data(samples_number : int, min : float, max : float, separation_vertical : float):
    """Generates data with 2 features and 2 classes separable by a vertical line"""
    features = []
    labels = []
    for i in range(samples_number):
        x = random.random() * (max - min) + min
        y = random.random() * (max - min) + min
        features.append([x, y])
        labels.append(0 if x < separation_vertical else 1)
    
    data = { 'Features' : features, 'Labels' : labels }
    return data

# Generate training and validation data using the same separation vertical
separation_vertical = 0.5
training_data_vertical = generate_vertically_separated_data(150, 0, 1, separation_vertical)
validation_data_vertical = generate_vertically_separated_data(50, 0, 1, separation_vertical)
print("Training and validation data generated")

# Set up lines that show class separation
class_separation_lines_vertical = [[[separation_vertical, separation_vertical], [0, 1]]]
    
plot_data(training_data_vertical['Features'], 
          training_data_vertical['Labels'], extra_lines=class_separation_lines_vertical)

### Feature Engineering: Split Fanout

We'll use feature engineering - adding new features to the data before encoding it into the circuit for training and classification. 

For these examples, we'll use _split fanout_ preprocessing, for which the resulting data is a tensor product of (concatenation of the left halves of parameters and original features) and (concatenation of the right halves). 
That is, 2 original features $[x_0, x_1]$ and 2 parameters $[a_0, a_1]$ produce 4 new features $[a_0a_1, a_0x_1, x_0a_1, x_0x_1]$ that will be encoded in a 2-qubit state $(a_1|0\rangle + x_1|1\rangle) \otimes (a_0|0\rangle + x_0|1\rangle)$.

In [ ]:
%%qsharp

// Split fanout is a tensor product of (concatenation of left halves of parameters and input) and (concatenation of right halves)
// [auxil[0] * auxil[1], auxil[0] * input[1], input[0] * auxil[1], input[0] * input[1]]
// the length of the result is len(auxil) * len(input)
function FeaturesSplitFanout (auxil : Double[], input : Double[]) : Double[] {
    let halfLa = Length(auxil) / 2;
    let halfLi = Length(input) / 2;
    let left = auxil[...(halfLa-1)] + input[...(halfLi-1)];
    let right = auxil[halfLa...] + input[halfLi...];

    mutable ret = [];
    for j in 0 .. Length(left) - 1 {
        for k in 0 .. Length(right) - 1 {
            set ret += [left[j] * right[k]];
        }
    }

    return ret;
}

// The definition of classifier structure for the case when the data is vertically separable
function ClassifierStructureVerticalData() : ControlledRotation[] {
    return [
        ControlledRotation((0, []), PauliY, 0)
    ];
}

// The training code for the case when the data is vertically separable
operation TrainModelVerticalData(
    trainingVectors : Double[][],
    trainingLabels : Int[],
    featureEngineeringParameters : Double[],
    initialParameters : Double[][]
) : (Double[], Double) {
    let engineeredData = Mapped(FeaturesSplitFanout(featureEngineeringParameters, _), trainingVectors);
    return TrainClassifierModel(engineeredData, trainingLabels, ClassifierStructureVerticalData(), initialParameters);
}

// The classification code for the case when the data is linearly separable by lines going through the origin.
operation ClassifyVerticalData(
    samples : Double[][],
    featureEngineeringParameters : Double[],
    parameters : Double[],
    bias : Double
) : Int[] {
    let engineeredData = Mapped(FeaturesSplitFanout(featureEngineeringParameters, _), samples);
    return ClassifyData(engineeredData, ClassifierStructureVerticalData(), parameters, bias);
}


### <span style="color:blue">Exercise 1.</span> Figure out feature engineering parameters for classes separable by a vertical line

In the code cell below, replace `...` with the parameters $[a_0, a_1]$ used with split fanout preprocessing to produce the data for training and classification.

In [ ]:
# TODO: figure out parameters for split fanout feature engineering
feature_engineering_params_vertical = [...]

# Train the model
(parameters, bias) = TrainModelVerticalData.simulate(
    trainingVectors=training_data_vertical['Features'],
    trainingLabels=training_data_vertical['Labels'],
    featureEngineeringParameters=feature_engineering_params_vertical,
    initialParameters=[[0.4], [0.6]]
)

In [ ]:
# Classify validation data set using training results
classified_labels = ClassifyVerticalData.simulate(
    samples=validation_data_vertical['Features'],
    featureEngineeringParameters=feature_engineering_params_vertical,
    parameters=parameters, bias=bias
)

# Calculate miss rate
mask = np.not_equal(validation_data_vertical['Labels'], classified_labels)
miss_count = np.array(classified_labels)[np.where(mask)].size
miss_rate = miss_count / len(classified_labels)
print(f"Miss rate: {miss_rate:0.2%}")

# Plot validation results
plot_data(validation_data_vertical['Features'], validation_data_vertical['Labels'], classified_labels, class_separation_lines_vertical)

### 2.2: Classes Separable by Horizontal Lines

Let's repeat the same steps for two classes separable by horizontal lines.

In [ ]:
def generate_horizontally_separated_data(samples_number : int, min : float, max : float, separation_horizontal : float):
    """Generates data with 2 features and 2 classes separable by horizontal lines"""
    features = []
    labels = []
    for i in range(samples_number):
        x = random.random() * (max - min) + min
        y = random.random() * (max - min) + min
        features.append([x, y])
        labels.append(0 if abs(y) < separation_horizontal else 1)
    
    data = { 'Features' : features, 'Labels' : labels }
    return data

# Generate training and validation data using the same separation horizontals
separation_horizontal = 0.75
training_data_horizontal = generate_horizontally_separated_data(150, -1.5, 1.5, separation_horizontal)
validation_data_horizontal = generate_horizontally_separated_data(50, -1.5, 1.5, separation_horizontal)
print("Training and validation data generated")

# Set up lines that show class separation
class_separation_lines_horizontal = [[[-1.5, 1.5], [separation_horizontal, separation_horizontal]],
                                     [[-1.5, 1.5], [-separation_horizontal, -separation_horizontal]]]
    
plot_data(training_data_horizontal['Features'], training_data_horizontal['Labels'], extra_lines=class_separation_lines_horizontal)

In [ ]:
%%qsharp

// The definition of classifier structure for the case when the data is horizontally separable.
// Three controlled Ry gates with the same parameter approximate a SWAP gate
function ClassifierStructureHorizontalData() : ControlledRotation[] {
    return [ControlledRotation((0, [1]), PauliY, 0),
            ControlledRotation((1, [0]), PauliY, 0),
            ControlledRotation((0, [1]), PauliY, 0)];
}

// The training code for the case when the data is horizontally separable
operation TrainModelHorizontalData(
    trainingVectors : Double[][],
    trainingLabels : Int[],
    featureEngineeringParameters : Double[],
    initialParameters : Double[][]
) : (Double[], Double) {
    let engineeredData = Mapped(FeaturesSplitFanout(featureEngineeringParameters, _), trainingVectors);
    return TrainClassifierModel(engineeredData, trainingLabels, ClassifierStructureHorizontalData(), initialParameters);
}

// The classification code for the case when the data is linearly separable by lines going through the origin.
operation ClassifyHorizontalData(
    samples : Double[][],
    featureEngineeringParameters : Double[],
    parameters : Double[],
    bias : Double
) : Int[] {
    let engineeredData = Mapped(FeaturesSplitFanout(featureEngineeringParameters, _), samples);
    return ClassifyData(engineeredData, ClassifierStructureHorizontalData(), parameters, bias);
}


### <span style="color:blue">Exercise 2.</span> Figure out feature engineering parameters for classes separable by horizontal lines

In the code cell below, replace `...` with the parameters $[a_0, a_1]$ used with split fanout preprocessing to produce the data for training and classification.

In [ ]:
# TODO: figure out parameters for split fanout feature engineering
feature_engineering_params_horizontal = [...]

# Train the model
(parameters, bias) = TrainModelHorizontalData.simulate(
    trainingVectors=training_data_horizontal['Features'],
    trainingLabels=training_data_horizontal['Labels'],
    featureEngineeringParameters=feature_engineering_params_horizontal,
    initialParameters=[[1.0], [2.0], [3.0]]
)

In [ ]:
# Classify validation data set using training results
classified_labels = ClassifyHorizontalData.simulate(
    samples=validation_data_horizontal['Features'],
    featureEngineeringParameters=feature_engineering_params_horizontal,
    parameters=parameters, bias=bias
)

# Calculate miss rate
mask = np.not_equal(validation_data_horizontal['Labels'], classified_labels)
miss_count = np.array(classified_labels)[np.where(mask)].size
miss_rate = miss_count / len(classified_labels)
print(f"Miss rate: {miss_rate:0.2%}")

# Plot validation results
plot_data(validation_data_horizontal['Features'], validation_data_horizontal['Labels'], classified_labels, class_separation_lines_horizontal)

## 3. Classes Separable by Second Degree Curves

### 3.1. Classes Separable by a Circle

In [ ]:
def generate_circle_separated_data(samples_number : int, min : float, max : float, separation_r : float):
    """Generates data with 2 features and 2 classes separable by a circle"""
    features = []
    labels = []
    for i in range(samples_number):
        x = random.random() * (max - min) + min
        y = random.random() * (max - min) + min
        features.append([x, y])
        labels.append(0 if x**2 + y**2 < separation_r**2 else 1)
    
    data = { 'Features' : features, 'Labels' : labels }
    return data

# Generate training and validation data using the same separation circle
separation_r = 0.8
training_data_circle = generate_circle_separated_data(150, -1, 1, separation_r)
validation_data_circle = generate_circle_separated_data(50, -1, 1, separation_r)
print("Training and validation data generated")

plot_data(training_data_circle['Features'], training_data_circle['Labels'])
pyplot.gca().add_patch(pyplot.Circle((0, 0), 0.8, color='gray', fill=False))

### Feature Engineering: Left Padding

For these examples, we'll use _left padding_ preprocessing, for which the resulting data is a concatenation of the parameters array and the original features array. 
That is, 
* 2 original features $[x_0, x_1]$ and 1 parameter $[a_0]$ produce 3 new features $[a_0, x_0, x_1]$ that will be encoded in a 2-qubit state $a_0|00\rangle + x_0|10\rangle + x_1|01\rangle$.
* 2 original features $[x_0, x_1]$ and 2 parameters $[a_0, a_1]$ produce 4 new features $[a_0, a_1, x_0, x_1]$ that will be encoded in a 2-qubit state $a_0|00\rangle + a_1|10\rangle + x_0|01\rangle + x_1|11\rangle$.

In [ ]:
%%qsharp

// Left padding preprocessing is a concatenation of the parameters array and the original features array. 
// The length of the result is len(auxil) + len(input)
function LeftPaddedFeatures(auxil : Double[], input : Double[]) : Double[] {
    return auxil + input;
}

// The definition of classifier structure for the case when the data is separable by a circle
function ClassifierStructureCircleData() : ControlledRotation[] {
    return [
        ControlledRotation((0, []), PauliY, 0)
    ];
}

// The training code for the case when the data is separable by a circle
operation TrainModelCircleData(
    trainingVectors : Double[][],
    trainingLabels : Int[],
    featureEngineeringParameters : Double[],
    initialParameters : Double[][]
) : (Double[], Double) {
    let engineeredData = Mapped(LeftPaddedFeatures(featureEngineeringParameters, _), trainingVectors);
    return TrainClassifierModel(engineeredData, trainingLabels, ClassifierStructureCircleData(), initialParameters);
}

// The classification code for the case when the data is separable by a circle
operation ClassifyCircleData(
    samples : Double[][],
    featureEngineeringParameters : Double[],
    parameters : Double[],
    bias : Double
) : Int[] {
    let engineeredData = Mapped(LeftPaddedFeatures(featureEngineeringParameters, _), samples);
    return ClassifyData(engineeredData, ClassifierStructureCircleData(), parameters, bias);
}


### <span style="color:blue">Exercise 3.</span> Figure out feature engineering parameters for classes separable by a circle

In the code cell below, replace `...` with the parameters $[a_0]$ or $[a_0, a_1]$ used with left padding preprocessing to produce the data for training and classification.

In [ ]:
# TODO: figure out parameters for left padding feature engineering
feature_engineering_params_circle = [...]

# Train the model
(parameters, bias) = TrainModelCircleData.simulate(
    trainingVectors=training_data_circle['Features'],
    trainingLabels=training_data_circle['Labels'],
    featureEngineeringParameters=feature_engineering_params_circle,
    initialParameters=[[1.0], [2.0], [3.0]]
)

In [ ]:
# Classify validation data set using training results
classified_labels = ClassifyCircleData.simulate(
    samples=validation_data_circle['Features'],
    featureEngineeringParameters=feature_engineering_params_circle,
    parameters=parameters, bias=bias
)

# Calculate miss rate
mask = np.not_equal(validation_data_circle['Labels'], classified_labels)
miss_count = np.array(classified_labels)[np.where(mask)].size
miss_rate = miss_count / len(classified_labels)
print(f"Miss rate: {miss_rate:0.2%}")

# Plot validation results
plot_data(validation_data_circle['Features'], validation_data_circle['Labels'], classified_labels)
pyplot.gca().add_patch(pyplot.Circle((0, 0), 0.8, color='gray', fill=False))

### 3.2. Classes Separable by a Hyperbola

In [ ]:
def generate_hyperbola_separated_data(samples_number : int, min : float, max : float, separation_r : float):
    """Generates data with 2 features and 2 classes separable by a hyperbola"""
    features = []
    labels = []
    for i in range(samples_number):
        x = random.random() * (max - min) + min
        y = random.random() * (max - min) + min
        features.append([x, y])
        labels.append(0 if separation_r**2 + x**2 > y**2 else 1)
    
    data = { 'Features' : features, 'Labels' : labels }
    return data

# Generate training and validation data using the same separation circle
separation_r = 0.3
training_data_hyperbola = generate_hyperbola_separated_data(150, -1, 1, separation_r)
validation_data_hyperbola = generate_hyperbola_separated_data(50, -1, 1, separation_r)
print("Training and validation data generated")

plot_data(training_data_hyperbola['Features'], training_data_hyperbola['Labels'])

In [ ]:
%%qsharp

// The definition of classifier structure for the case when the data is separable by a hyperbola
function ClassifierStructureHyperbolaData() : ControlledRotation[] {
    return [ControlledRotation((0, []), PauliY, 0)];
}

// The training code for the case when the data is separable by a hyperbola
operation TrainModelHyperbolaData(
    trainingVectors : Double[][],
    trainingLabels : Int[],
    featureEngineeringParameters : Double[],
    initialParameters : Double[][]
) : (Double[], Double) {
    let engineeredData = Mapped(LeftPaddedFeatures(featureEngineeringParameters, _), trainingVectors);
    return TrainClassifierModel(engineeredData, trainingLabels, ClassifierStructureHyperbolaData(), initialParameters);
}

// The classification code for the case when the data is separable by a hyperbola
operation ClassifyHyperbolaData(
    samples : Double[][],
    featureEngineeringParameters : Double[],
    parameters : Double[],
    bias : Double
) : Int[] {
    let engineeredData = Mapped(LeftPaddedFeatures(featureEngineeringParameters, _), samples);
    return ClassifyData(engineeredData, ClassifierStructureHyperbolaData(), parameters, bias);
}


### <span style="color:blue">Exercise 4.</span> Figure out feature engineering parameters for classes separable by a hyperbola

In the code cell below, replace `...` with the parameters $[a_0]$ or $[a_0, a_1]$ used with left padding preprocessing to produce the data for training and classification.

In [ ]:
# TODO: figure out parameters for left padding feature engineering
feature_engineering_params_hyperbola = [...]

# Train the model
(parameters, bias) = TrainModelHyperbolaData.simulate(
    trainingVectors=training_data_hyperbola['Features'],
    trainingLabels=training_data_hyperbola['Labels'],
    featureEngineeringParameters=feature_engineering_params_hyperbola,
    initialParameters=[[1.0], [2.0], [3.0]]
)

In [ ]:
# Classify validation data set using training results
classified_labels = ClassifyHyperbolaData.simulate(
    samples=validation_data_hyperbola['Features'],
    featureEngineeringParameters=feature_engineering_params_hyperbola,
    parameters=parameters, bias=bias
)

# Calculate miss rate
mask = np.not_equal(validation_data_hyperbola['Labels'], classified_labels)
miss_count = np.array(classified_labels)[np.where(mask)].size
miss_rate = miss_count / len(classified_labels)
print(f"Miss rate: {miss_rate:0.2%}")

# Plot validation results
plot_data(validation_data_hyperbola['Features'], validation_data_hyperbola['Labels'], classified_labels)

## What's Next?

This tutorial covered classifying artificial data, taking advantage of its simple structure. Classifying real data will require more complex models - same as in traditional machine learning.

* Check out [introduction to quantum machine learning](https://docs.microsoft.com/azure/quantum/user-guide/libraries/machine-learning/) at Microsoft Quantum Development Kit documentation, which features a more interesting example - classifying half-moons dataset.
* [Quantum machine learning samples](https://github.com/microsoft/Quantum/tree/main/samples/machine-learning) offer examples of classifying several more datasets.